In [1]:
from sqlalchemy.index import DatabaseHandler

ImportError: cannot import name 'create_engine' from 'sqlalchemy' (/home/felipe/adatech/data-engineer/analytics_engineering/projeto/sqlalchemy/__init__.py)